# Using the PlayerData API with Pydantic

This notebook demonstrates how to use the PlayerData API with Pydantic to query session details and raw data.

## Prerequisites

Before using this notebook, you will need to:
- Create a virtual environment
- Install all dependencies from the `pyproject.toml` file

## Authentication

First, we need to authenticate with the PlayerData API. There are three authentication methods available:

1. **Client Credentials Flow** (shown in this example)
2. **Authorization Code Flow**
3. **Authorization Code Flow with PKCE**

To use a different authentication method, change the `authentication_type` parameter to `AuthenticationType.AUTHORIZATION_CODE_FLOW` or `AuthenticationType.AUTHORIZATION_CODE_FLOW_WITH_PKCE` and provide the required parameters.

In [ ]:
from playerdatapy.playerdata_api import PlayerDataAPI
from playerdatapy.gqlauth import AuthenticationType

# Add your client id, client secret and one of your club ids
CLIENT_ID = "you_client_id"
CLIENT_SECRET = "you_client_secret"
CLUB_ID = "you_club_id"

# Create a PlayerDataAPI instance for authentication
api = PlayerDataAPI(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    authentication_type=AuthenticationType.CLIENT_CREDENTIALS_FLOW,
)

## Defining Queries

There are example queries in the `queries` folder that you can use as a starting point. For example, `last_thirty_days_sessions` retrieves all sessions for a club in the last 30 days.

You can also define your own queries using the `Query` class and the generated schema. These models are generated using [ariadne-codegen](https://github.com/mirumee/ariadne-codegen/tree/main), which uses the GraphQL schema to generate Python models.

### Model Hierarchy

The models in the `playerdatapy` package follow this hierarchy:

1. **BaseModel** (`playerdatapy.base_model.BaseModel`)
   - Inherits from Pydantic's `BaseModel`
   - Base class for all data models
   - Provides configuration for field aliasing, validation, and arbitrary types

2. **Input Types** (`playerdatapy.input_types.*`)
   - All inherit from `BaseModel`
   - Used for GraphQL input arguments (mutations, query filters, etc.)
   - Examples: `MutateSessionAttributes`, `SessionsSessionFilter`, `SessionParticipationAttributes`

3. **GraphQLField** (`playerdatapy.base_operation.GraphQLField`)
   - Base class for building GraphQL field selections
   - Used to construct GraphQL queries and mutations programmatically

4. **Field Classes** (`playerdatapy.custom_fields.*`)
   - All inherit from `GraphQLField`
   - Represent GraphQL response types and field selections
   - Examples: `AthleteFields`, `SessionInterface`, `TrainingSessionFields`, `MatchSessionFields`

5. **Query Class** (`playerdatapy.custom_queries.Query`)
   - Contains class methods that return `GraphQLField` instances
   - Methods return specific field classes from `custom_fields`
   - Examples: `Query.sessions()`, `Query.athlete()`, `Query.club()`

6. **Mutation Class** (`playerdatapy.custom_mutations.Mutation`)
   - Similar to Query, contains mutation methods
   - Methods return payload field classes (e.g., `CreateSessionPayloadFields`)
   - Examples: `Mutation.create_session()`, `Mutation.update_session()`

7. **Enums** (`playerdatapy.enums.*`)
   - Python enums representing GraphQL enum types
   - Examples: `SessionTypeEnum`, `MatchSessionResult`, `OrderDirectionEnum`


## Running Example Queries

For now, let's use the example queries, run them, and print the results.

**Note:** In the examples below, we use `await` directly since asyncio doesn't work simply in notebooks. For a complete asyncio example, see the `example_use.py` file.

In [ ]:
from queries.club_sessions_filtered_by_time_range import (
    club_sessions_filtered_by_time_range,
)
from datetime import datetime, timedelta, timezone

# Query for all sessions in the last 30 days
start_date = datetime.now(timezone.utc) - timedelta(days=30)
end_date = datetime.now(timezone.utc)

last_thirty_days_sessions_response = await api.run_queries(
    "ClubSessionsFilteredByTimeRangeQuery",
    club_sessions_filtered_by_time_range(CLUB_ID, start_date, end_date),
)
print(last_thirty_days_sessions_response)

## Getting Session Details

From the response above, we can see the sessions returned. Now let's get detailed information for a specific session using the session ID from the first session in the response.

In [ ]:
from queries.session_details import session_details

# Extract the session ID from the first session
first_session_id = last_thirty_days_sessions_response["sessions"][2]["id"]

# Query for detailed session information
session_details_response = await api.run_queries(
    "SessionDetailsQuery", session_details(first_session_id)
)
print(session_details_response)

## Getting Session Metrics

For the same session, you can retrieve configured metrics at different levels:
- Team aggregate level
- Session participation level
- Segment level

In [ ]:
from queries.session_metrics import session_metrics

# Query for session metrics at all levels
session_metrics_response = await api.run_queries(
    "SessionMetricsQuery", session_metrics(first_session_id)
)
print(session_metrics_response)

## Advanced Usage: Raw Data

To query for raw data, you can use the `session_participation_urls` query. This returns URLs for the raw data for a given list of session participation IDs.

We'll use the first session participation ID from the `session_details_response`.

In [ ]:
from queries.session_participations_urls import session_participations_urls

# Extract the first session participation ID
print(session_details_response)
first_session_participation_id = session_details_response["session"][
    "sessionParticipations"
][0]["id"]

# Query for raw data URLs
session_participation_urls_response = await api.run_queries(
    "SessionParticipationUrlsQuery",
    session_participations_urls([first_session_participation_id]),
)
print(session_participation_urls_response)

## Downloading Raw Data

The response contains a list of URLs for the raw data for the given session participation ID.

You can download the raw JSON data in two ways:
1. Use the `url_to_csv` function from `raw_data_utils.url_to_csv.py` (shown below)
2. Use the `requests` library directly

The `url_to_csv` function downloads the raw JSON data from the URL and saves it to CSV files:
- One file for GPS data
- One file for IMU acceleration data
- One file for IMU orientation data

The session participation ID is used as a prefix in the filename to differentiate between files.

In [ ]:
from raw_data_utils.url_to_csv import url_to_csv

# Extract the URL from the first session participation
first_session_participation_url = session_participation_urls_response[
    "sessionParticipations"
][0]["datafiles"][0]["url"]

# Download and convert raw data to CSV files
url_to_csv(first_session_participation_url, first_session_participation_id)

This will create a folder in your root directory with the session participation ID and the three CSV files saved inside.

**Note:** This method of retrieving raw data via the API is currently only available for GPS and IMU data. This method will be deprecated in the future, making way for a simpler method of raw data retrieval where raw LPS data will also be available.